In [1]:
from Firefly.data_reader import Reader,ParticleGroup

## Creating a Custom Reader

### Understanding the Reader keyword arguments
`JSONdir=None` - This should be the name of the sub-directory that will contain your JSON files, if you are not running python from `/path/to/Firefly/data` it should be the absolute path.

`options=None` - An `Options` instance, if you have created one you can pass it here. `None` will generate default options. `reader.options.listKeys()` will give you a list of the different available options you can set using `reader.options["option_name"] = option_value`. 

`write_startup='append'` - This is a flag for whether `startup.json` file should be written. It has 3 values: `True` -> writes a new `startup.json` that will contain only this visualization, `'append'` -> which will add this visualization to an existing `startup.json` (or create a new one), this is the default option, or `False` -> which will not add an entry to `startup.json`.

`max_npart_per_file=10000` - The maximum number of particles saved per file, don't use too large a number or you will have trouble loading the individual files in. 

`prefix='Data'` - What you would like your `.json` files to be called when you run `reader.dumpToJSON`. The format is `(prefix)(particleGroupName)(fileNumber).json`.

`clean_JSONdir=0` - Whether you would like to delete all `.json` files in the `JSONdir`. Usually not necessary (since `filenames.json` will be updated) but good to clean up after yourself.

In [2]:
my_reader = Reader(
    ## the name of the sub-directory that will contain your JSON files,
    ##  if you are not running python from /path/to/Firefly/data it should be the absolute path
    JSONdir = '/Users/agurvich/research/repos/Firefly/data/MyReader', 
    ## options object, if you have one you can pass it, None will generate default options, see
    ##  below for options arguments/capabilities
    options = None, 
    ## whether a startup.json file should be written, defaults to 'append'
    write_startup = True,
    max_npart_per_file= 10**4, 
    prefix = 'MyData',
    clean_JSONdir = False
)


## Loading in (or Creating) your Custom Data

In [3]:
## Here I'll create a grid of points in the shape of a cube
my_coords = np.linspace(-10,10,20)
xs,ys,zs = np.meshgrid(my_coords,my_coords,my_coords)
xs,ys,zs = xs.flatten(),ys.flatten(),zs.flatten()
coords = np.array([xs,ys,zs]).T
print(coords.shape)

(8000, 3)


## Store the coordinates in a ParticleGroup

### Understanding the ParticleGroup keyword arguments
`UIname` - Name of the particle group that shows up in the UI, 4-5 characters is best

`coordinates` - The coordinates of the points in 3d space, should have a shape of `(nparts,3)`.

`tracked_arrays=[]` - The arrays to associate with each coordinate in space, each array should be one-dimensional and have `nparts` entries.

`tracked_names=[]` - Should be the same length as `tracked_arrays`, and gives a name to each of the arrays when they show up in the UI dropdowns.

`tracked_filter_flags=[]` - Should be the same length as `tracked_arrays`, and gives a flag for whether that array should be available as an interactive filter within Firefly.

`tracked_colormap_flags=[]` - Should be the same length as `tracked_arrays`, and gives a flag for whether that array should be available as an interactive filter within Firefly.

`decimation_factor=1` - An integer factor to sub-sample the provided dataset at (in addition to any manual subsampling you might do). This will choose `nparts/decimation_factor` many points at random from the dataset to display in Firefly. 

`filenames_and_nparts=None` - Allows you to manually control how the particles are distributed among the JSON files, **highly recommended that you leave this to** `None`, but if for whatever reason you need fine-tuning you should pass a list of tuples in the form `[("json_name0.json",nparts_this_file0),("json_name1.json",nparts_this_file1) ... ]` where where the sum of `nparts_this_file%d` is exactly `nparts`. These files will automatically be added to `filenames.json` if you use `reader.dumpToJSON`.

`**option_kwargs` - allows you to set default options like the color, particle sizes, etc... for this particle group at the creation of the instance. You can see available options by looking at `list(particleGroup.options_default.keys())`.

In [4]:
my_parts = ParticleGroup(
    'partname',
    coords,
    ## below here I pass a few option_kwargs to set the size and color of the points
    sizeMult=5, ## make the particles a bit easier to see since there's so few of them
    color = [0,0,1,1], ## make them blue, I like blue. colors should be RGBA lists
    )

## Attaching your data to the reader

In [5]:
## tell the reader to keep track of this particle group
my_reader.addParticleGroup(my_parts)


[Particle Group of partname
 Contains 8000 (8000 after dec) particles and 0 arrays]

## Outputting to JSON

### Understanding Firefly's output requirements
Firefly (currently) uses `JSON` to load data into the browser's javascript interpreter. There are four classes of `.json` file that Firefly requires:

* `startup.json` - this is a specifically formatted configuration file that tells Firefly what dataset/visualization to load at startup. If it is in the form of an array Firefly will allow the user to select the dataset/visualization from a dropdown when Firefly is initialized. This must be contained within `Firefly/data`.

* `(options).json` - this is a `.json` file that contains the default options for various aspects of Firefly's UI, as well as default filter settings for each of the particle groups. It should be created using an Options instance and its corresponding `outputToJSON()` method. This lives in the `JSONdir` sub-directory described above. It can be named whatever you'd like as long as it is linked to correctly within `filenames.json`, described below.

* `(prefix)(particleName)(fileNumber).json` - These are the main data files, and contain the particle coordinates and their corresponding array values. They should be created using a ParticleGroup instance's `outputToJSON()` method. These files live in the `JSONdir` sub-directory described above. They can be named whatever you'd like as long as they are linked to correctly within `filenames.json`, described below.

* `filenames.json` - this is a specifically formatted configuration file that tells Firefly what `options.json` file to load and which data `.jsons` to load. This lives in the `JSONdir` sub-directory described above. The easiest way to create this file is to use a Reader instance's `dumpToJSON()` method that contains corresponding Options and ParticleGroup instances (see above). 

In [6]:
## have the reader dump all its data to various JSON files
my_reader.dumpToJSON()

outputting Particle Group of partname
Contains 8000 (8000 after dec) particles and 0 arrays

[] []
